In [ ]:
import pandas as pd

In [ ]:
trades = pd.read_html('./Statement.htm')
df = trades[0]

In [ ]:
# Checking where the header row starts
for index, row in df.iterrows():
    if row[0] == 'Ticket':
        header_row_index = index
        break


In [ ]:
# Set header row
new_header = df.iloc[header_row_index]
df2 = df[header_row_index+1:]
df2.columns = new_header
df2.reset_index(drop=True, inplace=True)

In [ ]:
# Find the last row of trades and remove everything below it
mask = df2[df2.columns[0:10]].isna().all(axis=1)
first_nan_index = mask.idxmax() if mask.any() else len(df2)
first_nan_index = mask.idxmax() if mask.any() else len(df2)
df_final = df2.iloc[:first_nan_index]

In [ ]:
# Remove deposits and withdrawals from df to get only pnl. 
df_deposit_withdrawal = df_final[df_final['Type'].str.lower().isin(['balance'])]
df_final = df_final[~df_final['Type'].str.lower().isin(['balance'])]
# Removed cancelled orders/trades
df_final = df_final[df_final['Profit'].str.lower() != 'cancelled']



In [ ]:
# Correct Dtypes for calculations later on. Also remove white spaces in Profit column.
df_final['Ticket'] = df_final['Ticket'].astype(int)
cols_to_float = ['Size', 'Price', 'S / L', 'T / P', 'Price', 'Commission', 'Taxes', 'Swap' ,'Profit']

# Remove white spaces in Profit to convert from str to float
for col in cols_to_float:
    df_final[col] = df_final[col].replace(' ', '', regex=True).astype(float)

conversion_dict = {col: float for col in cols_to_float}
df = df_final.astype(conversion_dict)

In [ ]:
df_final.tail()